# Python 知识

# 代码分析

## 类继承关系
```mermaid
classDiagram
    class Configured
    class PandasIndexer
    class ColumnGrouper
    class ArrayWrapper
    class Wrapping
    
    Configured <|-- ArrayWrapper
    PandasIndexer <|-- ArrayWrapper
    Configured <|-- Wrapping
    PandasIndexer <|-- Wrapping
    AttrResolver <|-- Wrapping

    ArrayWrapper o-- ColumnGrouper : has
    
```

## class ArrayWrapper(Configured, PandasIndexer)

### `__init__`
```python
    def __init__(self,
                 index: tp.IndexLike,                           # 行索引，支持类似pandas索引的对象
                 columns: tp.IndexLike,                         # 列索引，支持类似pandas索引的对象
                 ndim: int,                      # 数组维度数量，通常为1（Series）或2（DataFrame）
                 freq: tp.Optional[tp.FrequencyLike] = None,    # 时间序列频率，可选
                 column_only_select: tp.Optional[bool] = None,  # 是否仅对列进行索引，可选
                 group_select: tp.Optional[bool] = None,        # 是否对分组进行索引，可选
                 grouped_ndim: tp.Optional[int] = None,         # 分组后的维度数量，可选
                 **kwargs) -> None:                             # 传递给ColumnGrouper的额外参数
        config = dict(
            index=index,
            columns=columns,
            ndim=ndim,
            freq=freq,
            column_only_select=column_only_select,
            group_select=group_select,
            grouped_ndim=grouped_ndim,
        )

        checks.assert_not_none(index)
        checks.assert_not_none(columns)
        checks.assert_not_none(ndim)
        
        if not isinstance(index, pd.Index):
            index = pd.Index(index) 
        if not isinstance(columns, pd.Index):
            columns = pd.Index(columns) 

        self._index = index 
        self._columns = columns
        self._ndim = ndim
        self._freq = freq
        self._column_only_select = column_only_select
        self._group_select = group_select
        self._grouper = ColumnGrouper(columns, **kwargs)
        self._grouped_ndim = grouped_ndim

        PandasIndexer.__init__(self) 
        Configured.__init__(self, **merge_dicts(config, self._grouper._config)) 
```

### indexing_func_meta

#### settings['array_wrapper']
```python
array_wrapper=dict(
    column_only_select=False,  # 不仅选择列
    group_select=True,  # 启用组选择
    freq=None,  # 频率默认为None
    silence_warnings=False  # 不静默警告
)
```

#### regroup
```python
def regroup(self: ArrayWrapperT, group_by: tp.GroupByLike, **kwargs) -> ArrayWrapperT:
    """
    重新分组对象
    
    根据新的分组依据创建新的ArrayWrapper实例。只有在分组发生变化时
    才创建新实例，否则返回自身以保持缓存有效性。
    
    Args:
        group_by: 新的分组依据
        **kwargs: 传递给replace方法的额外参数
        
    Returns:
        ArrayWrapperT: 重新分组后的ArrayWrapper实例
        
    Examples:
        >>> wrapper = ArrayWrapper(
        ...     index=pd.Index(['A', 'B']),
        ...     columns=pd.Index(['X', 'Y', 'Z']),
        ...     ndim=2
        ... )
        >>> # 重新分组
        >>> grouped = wrapper.regroup(['G1', 'G1', 'G2'])
        >>> print(grouped.grouper.get_group_count())
        2
        
        # 取消分组
        >>> ungrouped = grouped.regroup(None)
        >>> print(ungrouped.grouper.is_grouped())
        False
    """
    if self.grouper.is_grouping_changed(group_by=group_by):
        self.grouper.check_group_by(group_by=group_by)
        grouped_ndim = None
        if self.grouper.is_grouped(group_by=group_by):
            if not self.grouper.is_group_count_changed(group_by=group_by):
                grouped_ndim = self.grouped_ndim
        return self.replace(grouped_ndim=grouped_ndim, group_by=group_by, **kwargs)
    return self  # important for keeping cache
```


```python
@cached_method
def indexing_func_meta(self: ArrayWrapperT,
                        pd_indexing_func: tp.PandasIndexingFunc,
                        index: tp.Optional[tp.IndexLike] = None,
                        columns: tp.Optional[tp.IndexLike] = None,
                        column_only_select: tp.Optional[bool] = None,
                        group_select: tp.Optional[bool] = None,
                        group_by: tp.GroupByLike = None) -> IndexingMetaT:
    from vectorbt._settings import settings
    array_wrapper_cfg = settings['array_wrapper']

    if column_only_select is None:
        column_only_select = self.column_only_select
    if column_only_select is None:
        column_only_select = array_wrapper_cfg['column_only_select']
    if group_select is None:
        group_select = self.group_select
    if group_select is None:
        group_select = array_wrapper_cfg['group_select']
    _self = self.regroup(group_by)
    group_select = group_select and _self.grouper.is_grouped()
    if index is None:
        index = _self.index
    if not isinstance(index, pd.Index):
        index = pd.Index(index)
    if columns is None:
        if group_select:
            columns = _self.grouper.get_columns()
        else:
            columns = _self.columns
    if not isinstance(columns, pd.Index):
        columns = pd.Index(columns)
    if group_select:
        # Groups as columns
        i_wrapper = ArrayWrapper(index, columns, _self.get_ndim())
    else:
        # Columns as columns
        i_wrapper = ArrayWrapper(index, columns, _self.ndim)
    n_rows = len(index)
    n_cols = len(columns)

    if column_only_select:
        if i_wrapper.ndim == 1:
            raise IndexingError("Columns only: This object already contains one column of data")
        try:
            col_mapper = pd_indexing_func(i_wrapper.wrap_reduced(np.arange(n_cols), columns=columns))
        except pd.core.indexing.IndexingError as e:
            warnings.warn("Columns only: Make sure to treat this object "
                            "as a Series of columns rather than a DataFrame", stacklevel=2)
            raise e
        if checks.is_series(col_mapper):
            new_columns = col_mapper.index
            col_idxs = col_mapper.values
            new_ndim = 2
        else:
            new_columns = columns[[col_mapper]]
            col_idxs = col_mapper
            new_ndim = 1
        new_index = index
        idx_idxs = np.arange(len(index))
    else:
        idx_mapper = pd_indexing_func(i_wrapper.wrap(
            np.broadcast_to(np.arange(n_rows)[:, None], (n_rows, n_cols)),
            index=index,
            columns=columns
        ))
        if i_wrapper.ndim == 1:
            if not checks.is_series(idx_mapper):
                raise IndexingError("Selection of a scalar is not allowed")
            idx_idxs = idx_mapper.values
            col_idxs = 0
        else:
            col_mapper = pd_indexing_func(i_wrapper.wrap(
                np.broadcast_to(np.arange(n_cols), (n_rows, n_cols)),
                index=index,
                columns=columns
            ))
            if checks.is_frame(idx_mapper):
                idx_idxs = idx_mapper.values[:, 0]
                col_idxs = col_mapper.values[0]
            elif checks.is_series(idx_mapper):
                one_col = np.all(col_mapper.values == col_mapper.values.item(0))
                one_idx = np.all(idx_mapper.values == idx_mapper.values.item(0))
                if one_col and one_idx:
                    # One index and one column selected, multiple times
                    raise IndexingError("Must select at least two unique indices in one of both axes")
                elif one_col:
                    # One column selected
                    idx_idxs = idx_mapper.values
                    col_idxs = col_mapper.values[0]
                elif one_idx:
                    # One index selected
                    idx_idxs = idx_mapper.values[0]
                    col_idxs = col_mapper.values
                else:
                    raise IndexingError
            else:
                raise IndexingError("Selection of a scalar is not allowed")
        new_index = index_fns.get_index(idx_mapper, 0)
        if not isinstance(idx_idxs, np.ndarray):
            # One index selected
            new_columns = index[[idx_idxs]]
        elif not isinstance(col_idxs, np.ndarray):
            # One column selected
            new_columns = columns[[col_idxs]]
        else:
            new_columns = index_fns.get_index(idx_mapper, 1)
        new_ndim = idx_mapper.ndim

    if _self.grouper.is_grouped():
        # Grouping enabled
        if np.asarray(idx_idxs).ndim == 0:
            raise IndexingError("Flipping index and columns is not allowed")

        if group_select:
            # Selection based on groups
            # Get indices of columns corresponding to selected groups
            group_idxs = col_idxs
            group_idxs_arr = reshape_fns.to_1d_array(group_idxs)
            group_start_idxs = _self.grouper.get_group_start_idxs()[group_idxs_arr]
            group_end_idxs = _self.grouper.get_group_end_idxs()[group_idxs_arr]
            ungrouped_col_idxs = get_ranges_arr(group_start_idxs, group_end_idxs)
            ungrouped_columns = _self.columns[ungrouped_col_idxs]
            if new_ndim == 1 and len(ungrouped_columns) == 1:
                ungrouped_ndim = 1
                ungrouped_col_idxs = ungrouped_col_idxs[0]
            else:
                ungrouped_ndim = 2

            # Get indices of selected groups corresponding to the new columns
            # We could do _self.group_by[ungrouped_col_idxs] but indexing operation may have changed the labels
            group_lens = _self.grouper.get_group_lens()[group_idxs_arr]
            ungrouped_group_idxs = np.full(len(ungrouped_columns), 0)
            ungrouped_group_idxs[group_lens[:-1]] = 1
            ungrouped_group_idxs = np.cumsum(ungrouped_group_idxs)

            return _self.replace(
                index=new_index,
                columns=ungrouped_columns,
                ndim=ungrouped_ndim,
                grouped_ndim=new_ndim,
                group_by=new_columns[ungrouped_group_idxs]
            ), idx_idxs, group_idxs, ungrouped_col_idxs

        # Selection based on columns
        col_idxs_arr = reshape_fns.to_1d_array(col_idxs)
        return _self.replace(
            index=new_index,
            columns=new_columns,
            ndim=new_ndim,
            grouped_ndim=None,
            group_by=_self.grouper.group_by[col_idxs_arr]
        ), idx_idxs, col_idxs, col_idxs

    # Grouping disabled
    return _self.replace(
        index=new_index,
        columns=new_columns,
        ndim=new_ndim,
        grouped_ndim=None,
        group_by=None
    ), idx_idxs, col_idxs, col_idxs
```

## class Wrapping(Configured, PandasIndexer, AttrResolver)